https://www.youtube.com/watch?v=DbESHcCoWbM&t=13s

In [9]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl

In [12]:
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64,10)
        
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        logits = self.l3(h2)
        return logits
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-2) 
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        
        # flatten
        b = x.size(0)
        x = x.view(b, -1)
        
        # forward
        logits = self(x)
        
        # compute the objective function
        J = self.loss(logits, y)
        
        return J
        
    def train_dataloader(self):
        train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
        #train, val = random_split(train_data, [55000, 5000])
        train_loader = DataLoader(train_data, batch_size=32)
        #val_loader = DataLoader(val, batch_size=32)
        return train_loader
    
model = ImageClassifier()
        

In [13]:
trainer = pl.Trainer(max_epochs = 10)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type             | Params
------------------------------------------
0 | l1   | Conv2d           | 76    
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | loss | CrossEntropyLoss | 0     
------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.020     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 784]